## Random forest regressor best params (this will continue 5 or 6 Random forest regression models from Initial modleing to see If I can get over .80 R2 score with random forest)

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pprint import pprint
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR



import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
filepath = '../../processed_data/all_processed.csv'
all_enviro = pd.read_csv(filepath)

In [ ]:
X = all_enviro[['total population',
 'ozone',
 'pm2.5',
 'diesel pm',
 'pesticides',
 'traffic',
 'cleanup sites',
 'groundwater threats',
 'haz. waste',
 'imp. water bodies',
 'solid waste',
 'pollution burden',
 'asthma',
 'low birth weight',
 'education',
 'linguistic isolation',
 'poverty',
 'pop. char. ',
 'drinking water',
 'tox. release',
 'unemployment',
 'ces_per',
 'cardiovascular disease',
 'housing burden',
 'est total',
 'est gen',
 'est cold',
 'est farm',
 'est other']]
y = all_enviro['asthma']

In [ ]:
X=X.fillna(X.median())
y=y.fillna(y.median())

In [ ]:
rf = RandomForestRegressor(random_state = 42)

In [ ]:
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state= 42, test_size=.2)

In [ ]:
rfr = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rfr.fit(X_train, y_train)

In [ ]:
print(f'Train accuracy {rfr.score(X_train,y_train)}')
print(f'Test accuracy {rfr.score(X_test,y_test)}')

### Best features in data set